This is my notebook file for playing around with new features, testing, etc.

In [190]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
%autoreload 2

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sodapy import Socrata
import datetime
import numpy as np
import matplotlib.dates as mdates
import data_processing as dp
import definitions
import data_viz as dv
pd.set_option("display.max_rows", None)
import plotly.express as px
import definitions
import kaleido

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import dash
from dash import dcc
from dash import html

In [142]:
dp.get_data()

LOG: Fetched all raw data
LOG: Processed CDC data
LOG: Filtered Data


LOG: Added HHS Provisional data
LOG: Got raw CDC data
LOG: Done getting data


In [ ]:
dv.generate(path="../")
# #dp.test_data
# dp.get_us_positivity()
# # dp.get_state_positivity(["CA"])
# print(dp.test_data)

In [ ]:
state_hosps = dp.get_all_state_hosps()
hosp_fig = px.choropleth(state_hosps, locations='State', locationmode="USA-states", color="Hospitalizations per Million", scope="usa", color_continuous_scale='YlOrRd')
hosp_fig.update_layout(
    title={
        'text': "USA Hospitalizations per Million",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title="")
        
hosp_fig.show()
pio.kaleido.scope.default_scale = 10
hosp_fig.write_image("../images/maps/hosps.jpeg")
# case_fig = px.choropleth(state_cases, locations='State', locationmode="USA-states", color="Cases", scope="usa", color_continuous_scale='Purples')
# case_fig.show()

In [ ]:
state_rt = dp.get_all_state_rt()
hosp_fig = px.choropleth(state_rt, locations='State', locationmode="USA-states", color="Rt", scope="usa", color_continuous_scale='YlOrRd')
hosp_fig.update_layout(
    title={
        'text': "USA Weekly Growth in Hospitalizations",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title="Rt")
pio.kaleido.scope.default_scale = 10
hosp_fig.write_image("../images/maps/rt.jpeg")

In [ ]:
state_cases = pd.DataFrame(columns=['State', 'Cases'])
for state in definitions.states.keys():
    state_data = dp.get_state_cases([state], start_date=(pd.Timestamp.today() - pd.Timedelta(days=7)).date()) # adjust end date here
    if state_data.empty:
        cases = False
    else:
        cases = state_data[state_data['date'] == state_data['date'].max()].cases.values
        
        cases = cases[0]/definitions.populations[definitions.states[state]]*1000000
        state_cases = state_cases.append({"State":state, "Cases": float(cases)}, ignore_index=True)
case_fig = px.choropleth(state_cases, locations='State', locationmode="USA-states", color="Cases", scope="usa", color_continuous_scale='Purples')

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1(children='Covid-Twitter-Bot-Dashboard'),

    html.Div(children='''
        Hospitalizations per capita
    ''', justify="center", align="center"),

    dcc.Graph(
    id='example-graph',
    figure=case_fig
    ),

    dcc.Graph(
    id='example-grap2h',
    figure=hosp_fig
    ),
    
])

if __name__ == '__main__':
    app.run_server()

In [68]:
import pandas as pd
from sodapy import Socrata
cdc_client = Socrata("data.cdc.gov", None)
cdc_state_raw = cdc_client.get("pwn4-m3yp", select="state, start_date, end_date, new_cases, new_deaths", limit=2000000)
cdc_state_df = pd.DataFrame.from_records(cdc_state_raw)
cdc_state_df.start_date = pd.to_datetime(cdc_state_df.start_date)
cdc_state_df.end_date = pd.to_datetime(cdc_state_df.end_date)
cdc_state_df.new_cases = cdc_state_df.new_cases.astype(float)
cdc_state_df.new_deaths = cdc_state_df.new_deaths.astype(float)


In [108]:
"""
get_state_data 
Creates dataframe of time series weekly date and data for given state
inputs:
 state_codes: List of 2-letter codes of states to query
 start_date (pd.Timestamp): starting date, defaults to 1-1-2020
 end_date (pd.Timestamp): ending date, defaults to today 
 normalize: boolean, whether to normalize by state population
 dataset: dataset to use, defaults to CDC dataset
 colName: Name of column containing data to add to dataframe
returns:
 df with 'week_start_date', 'week_end_date', and data
"""
def get_state_data_weekly(state_codes, start_date = pd.Timestamp(2020,1,1), end_date = pd.Timestamp.today(), normalize=True, dataset = cdc_state_df, colName = 'new_cases'):
    curr_date = start_date
    input_states = [definitions.states[s] for s in state_codes]
    state_data = dataset[dataset.state.isin(state_codes)][:]
    max_date = state_data.end_date.max()
    states_population = sum([definitions.populations[s] for s in input_states])
    lst = []
    while(curr_date <= end_date and curr_date <= max_date):
        if not (curr_date in state_data['end_date'].values):
            curr_date += datetime.timedelta(1)
            continue
        week_data = state_data[state_data.end_date == curr_date] 
        if normalize:
            case_sum = week_data[colName].sum() / states_population * 1000000
        else:
            case_sum = week_data[colName].sum()
        newRow = {'week_start_date': curr_date - datetime.timedelta(6), 'week_end_date': curr_date, colName: case_sum}
        lst.append(newRow)
        curr_date += datetime.timedelta(1)
    return pd.DataFrame(lst)




In [193]:

# ca_cases = dp.get_state_data_weekly(['CA', 'IL'], normalize=False, dataset=cdc_state_df, colName='new_cases')
# usa_cases = dp.get_all_state_data_weekly(dataset=cdc_state_df, normalize=True, colName='new_cases')
# print(usa_cases)
newestData = dp.DataSets(from_csv = True)
# dv.plot_graphs()

FileNotFoundError: [Errno 2] No such file or directory: 'data/cdc.csv'

In [189]:
a = "hi"
dv.plot_graphs(region = "California", path = '../', data = newestData)

[autoreload of data_viz failed: Traceback (most recent call last):
  File "/Users/jonahdf/Local Documents/covid-twitter-bot/ctb/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jonahdf/Local Documents/covid-twitter-bot/ctb/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/opt/homebrew/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/jonahdf/Local Documents/covid-twitter-bot/src/data_viz.py", line 381, in <module>
    newestData = dp.DataSets(from_csv= True)
  File "/Users/jonahdf/Local Docu

TypeError: get_state_positivity() got multiple values for argument 'dataset'

In [23]:
cdc_state_df.to_csv("output.csv", index=False)

In [98]:
[definitions.states[s] for s in ['CA', 'IL']]

['California', 'Illinois']